# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install ffmpeg
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy swifter

# !pip install -q --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110
# !pip install -q --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-tf-plugin-cuda110

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
     |████████████████████████████████| 647 kB 6.1 MB/s 
     |████████████████████████████████| 280 kB 82.6 MB/s 
     |████████████████████████████████| 136 kB 83.2 MB/s 


In [3]:
import os
import glob
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf_v1
import logging
import numpy as np
import cupy as cp
import mediapy
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import cv2
from time import sleep
from time import time
import mediapy as media
import shutil

from tensorflow.keras import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D
from tensorflow.keras.layers import Reshape, AveragePooling2D 
from tensorflow.keras.layers import Dense, Dropout, Input 
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import LSTM, Multiply
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

from tensorflow import optimizers

tf.config.set_soft_device_placement(True)
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false'
tf.get_logger().setLevel(logging.ERROR)

## Teste

In [4]:
video = media.read_video('/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train/Robbery035_x264-a4f337e6cb_1_resized_blur.mp4')
video_tensor = tf.constant([video[:32]], dtype=tf.float32)

In [5]:
lstm = ConvLSTM2D(filters=256, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True)

result = lstm(video_tensor)
result.shape # (samples, timesteps, new_rows, new_cols, filters)

TensorShape([1, 32, 224, 224, 256])

In [6]:
rgb = video_tensor
rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

print('SHAPE BLOCO CONV 1: {0} = {1} values. '.format(str(rgb.shape), str(rgb.shape[0]*rgb.shape[1]*rgb.shape[2]*rgb.shape[3]*rgb.shape[4])))


#####################################################
x = MaxPooling3D(pool_size=(8,1,1))(rgb)
print('SHAPE MAX 2: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))
#####################################################
x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 3: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 4: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,3,3))(x)

print('SHAPE BLOCO CONV 5: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

SHAPE BLOCO CONV 1: (1, 32, 14, 14, 32) = 200704 values. 
SHAPE MAX 2: (1, 4, 14, 14, 32) = 25088 values. 
SHAPE BLOCO CONV 3: (1, 2, 7, 7, 64) = 6272 values. 
SHAPE BLOCO CONV 4: (1, 1, 3, 3, 64) = 576 values. 
SHAPE BLOCO CONV 5: (1, 0, 1, 1, 128) = 0 values. 


In [7]:
rgb = video_tensor
rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

print('SHAPE BLOCO CONV 1: {0} = {1} values. '.format(str(rgb.shape), str(rgb.shape[0]*rgb.shape[1]*rgb.shape[2]*rgb.shape[3]*rgb.shape[4])))


#####################################################
x = MaxPooling3D(pool_size=(8,1,1))(rgb)
print('SHAPE MAX 2: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))
#####################################################
"""x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 3: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 4: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,3,3))(x)

print('SHAPE BLOCO CONV 5: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))"""

#Em paralelo com o x de cima"


lstm = ConvLSTM2D(filters=32, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
lstm = ConvLSTM2D(filters=32, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)

lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
print('ConvLSTM2D 1: {0} = {1} values. '.format(str(lstm.shape), str(lstm.shape[0]*lstm.shape[1]*lstm.shape[2]*lstm.shape[3]*lstm.shape[4])))


lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)

lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
print('ConvLSTM2D 2: {0} = {1} values. '.format(str(lstm.shape), str(lstm.shape[0]*lstm.shape[1]*lstm.shape[2]*lstm.shape[3]*lstm.shape[4])))

lstm = ConvLSTM2D(filters=128, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
lstm = ConvLSTM2D(filters=128, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)

lstm = MaxPooling3D(pool_size=(2,3,3))(lstm)
print('ConvLSTM2D 3: {0} = {1} values. '.format(str(lstm.shape), str(lstm.shape[0]*lstm.shape[1]*lstm.shape[2]*lstm.shape[3]*lstm.shape[4])))

SHAPE BLOCO CONV 1: (1, 32, 14, 14, 32) = 200704 values. 
SHAPE MAX 2: (1, 4, 14, 14, 32) = 25088 values. 
ConvLSTM2D 1: (1, 2, 7, 7, 32) = 3136 values. 
ConvLSTM2D 2: (1, 2, 7, 7, 64) = 6272 values. 
ConvLSTM2D 3: (1, 2, 4, 4, 128) = 4096 values. 


In [8]:
# https://arxiv.org/pdf/1709.06531.pdf
# Learning to detect violence videos using convolutional long short-term memory. 2017.
# https://github.com/aggression-detect/aggression_detect/blob/eedbaf3ab15595998ea8af1620c237d68318a42f/LSTM_CNN/BuildModel_basic.py
rgb = video_tensor
rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

rgb = Conv3D(
    32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = Conv3D(
    32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

print('SHAPE BLOCO CONV 1: {0} = {1} values. '.format(str(rgb.shape), str(rgb.shape[0]*rgb.shape[1]*rgb.shape[2]*rgb.shape[3]*rgb.shape[4])))


#####################################################
x = MaxPooling3D(pool_size=(8,1,1))(rgb)
print('SHAPE MAX 2: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))
#####################################################
"""x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 3: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,2,2))(x)

print('SHAPE BLOCO CONV 4: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))

x = Conv3D(
    128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = Conv3D(
    128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
x = MaxPooling3D(pool_size=(2,3,3))(x)

print('SHAPE BLOCO CONV 5: {0} = {1} values. '.format(str(x.shape), str(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4])))"""

#Em paralelo com o x de cima"

# cnn = TimeDistributed(cnn)(input_layer)

lstm = ConvLSTM2D(filters=256, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
print('ConvLSTM2D 1: {0} = {1} values. '.format(str(lstm.shape), str(lstm.shape[0]*lstm.shape[1]*lstm.shape[2]*lstm.shape[3]*lstm.shape[4])))


#lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
lstm = Flatten()(lstm)
print('Flatten 1: {0} = x values. '.format(str(lstm.shape)))


dropout = 0.5
lstm = BatchNormalization()(lstm)
print('BatchNormalization 1: {0} = x values. '.format(str(lstm.shape)))


lstm = Dropout(dropout)(lstm)
print('Dropout 1: {0} = x values. '.format(str(lstm.shape)))


lstm = Dense(1000)(lstm)
print('Dense 1: {0} = x values. '.format(str(lstm.shape)))


lstm = Activation('relu')(lstm)
print('Activation: {0} = x values. '.format(str(lstm.shape)))


lstm = Dense(256)(lstm)
print('Dense 1: {0} = x values. '.format(str(lstm.shape)))


lstm = Dropout(dropout)(lstm)
print('Dropout 1: {0} = x values. '.format(str(lstm.shape)))

lstm = Activation('relu')(lstm)
print('Activation: {0} = x values. '.format(str(lstm.shape)))

lstm = Dense(10)(lstm)
print('Dense 1: {0} = x values. '.format(str(lstm.shape)))

lstm = Dropout(dropout)(lstm)
print('Dropout 1: {0} = x values. '.format(str(lstm.shape)))

lstm = Activation('relu')(lstm)
print('Activation: {0} = x values. '.format(str(lstm.shape)))

#activation = 'sigmoid'
#loss_func = 'binary_crossentropy'


#FALTA A ÚLTIMA CAMADA DENSA



"""if classes > 1:
    activation = 'softmax'
    loss_func = 'categorical_crossentropy'
predictions = Dense(classes,  activation=activation)(relu)


model = Model(inputs=input_layer, outputs=predictions)
optimizer = optimizer_class[0](lr=learning_rate, **optimizer_class[1])
model.compile(optimizer=optimizer, loss=loss_func,metrics=['acc'])

print(model.summary())"""

SHAPE BLOCO CONV 1: (1, 32, 14, 14, 32) = 200704 values. 
SHAPE MAX 2: (1, 4, 14, 14, 32) = 25088 values. 
ConvLSTM2D 1: (1, 4, 14, 14, 256) = 200704 values. 
Flatten 1: (1, 200704) = x values. 
BatchNormalization 1: (1, 200704) = x values. 
Dropout 1: (1, 200704) = x values. 
Dense 1: (1, 1000) = x values. 
Activation: (1, 1000) = x values. 
Dense 1: (1, 256) = x values. 
Dropout 1: (1, 256) = x values. 
Activation: (1, 256) = x values. 
Dense 1: (1, 10) = x values. 
Dropout 1: (1, 10) = x values. 
Activation: (1, 10) = x values. 


"if classes > 1:\n    activation = 'softmax'\n    loss_func = 'categorical_crossentropy'\npredictions = Dense(classes,  activation=activation)(relu)\n\n\nmodel = Model(inputs=input_layer, outputs=predictions)\noptimizer = optimizer_class[0](lr=learning_rate, **optimizer_class[1])\nmodel.compile(optimizer=optimizer, loss=loss_func,metrics=['acc'])\n\nprint(model.summary())"

In [9]:
x.shape

TensorShape([1, 4, 14, 14, 32])

In [10]:
32*14*14*32

200704

# Início

In [11]:
# Paths base
PATH_BASE = '/content/drive/MyDrive/ucf_experiments/'
PATH_CICLE = PATH_BASE + 'ciclo_experimental_1/'

# Path base de armazenamento dos augmentations
PATH_DATA = PATH_CICLE + 'data/'

# Dados de treino augmentados
#PATH_AUG_BASE = PATH_DATA + 'augmented_train/'
# Dados de teste redimensionados (visto que não popdem ser augmentados)
#PATH_TEST_RESIZED = PATH_DATA + 'resized_test/'

PATH_BLUR_CONVLSTM = PATH_DATA + 'data_blur_convlstm2/' #data_blur
PATH_BLUR = PATH_DATA + 'data_blur/' #data_blur

In [12]:
# Create target directory & all intermediate directories if don't exists
def create_dir(dirName):
  try:
      os.makedirs(dirName) 
      print(dirName)   
      print("Directory " , dirName ,  " Created ")
      return dirName
  except FileExistsError:
      print("Directory " , dirName ,  " already exists")  
      return dirName


PATH_BLUR_CONVLSTM_MODEL = create_dir(PATH_BLUR_CONVLSTM + 'model/')
PATH_BLUR_CONVLSTM_MODEL_CHECKPOINTS = create_dir(PATH_BLUR_CONVLSTM_MODEL + 'checkpoints/')
PATH_TENSORBOARD = create_dir(PATH_BLUR_CONVLSTM_MODEL + 'runs/')

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/model/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/model/checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/model/runs/  already exists


## Uniform Sample

In [13]:
PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_TRAIN = create_dir(PATH_BLUR_CONVLSTM + 'uniform_samples/train/')
print(PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_TRAIN)

PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_TEST = create_dir(PATH_BLUR_CONVLSTM + 'uniform_samples/test/')
print(PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_TEST)

PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_CHECK_TRAIN = create_dir(PATH_BLUR_CONVLSTM + 'uniform_samples/checkpoints_train/')
print(PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_CHECK_TRAIN)

PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_CHECK_TEST = create_dir(PATH_BLUR_CONVLSTM + 'uniform_samples/checkpoints_test/')
print(PATH_BLUR_CONVLSTM_UNIFORM_SAMPLE_CHECK_TEST)

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/train/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/train/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/test/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/test/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/checkpoints_train/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/checkpoints_train/
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/checkpoints_test/  already exists
/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/uniform_samples/ch

In [14]:
# Copiando os dados do experimento data_blur para este (data_blur_lstm)
# PATH_BLUR -> PATH_BLUR_CONVLSTM

source=PATH_BLUR + 'train_uniform_sample_rwf2000.txt'
destination=PATH_BLUR_CONVLSTM + 'train_uniform_sample_rwf2000.txt'
shutil.copyfile(source, destination)

source=PATH_BLUR + 'test_uniform_sample_rwf2000.txt'
destination=PATH_BLUR_CONVLSTM + 'test_uniform_sample_rwf2000.txt'
shutil.copyfile(source, destination)

'/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur_convlstm2/test_uniform_sample_rwf2000.txt'

In [15]:
train_dataset = pd.read_csv(PATH_BLUR_CONVLSTM + 'train_uniform_sample_rwf2000.txt', sep=';', names=['video_path', 'label', 'new_path'])
test_dataset = pd.read_csv(PATH_BLUR_CONVLSTM + 'test_uniform_sample_rwf2000.txt', sep=';', names=['video_path', 'label', 'new_path'])

In [16]:
"""train_dataset = train_dataset.loc[
                                  (train_dataset.video_path.str.contains('_x264_') |
                                   ~train_dataset.video_path.str.contains('_x264-')) & 
                                  (train_dataset.video_path.str.contains('resized'))]

test_dataset = test_dataset.loc[
                                (test_dataset.video_path.str.contains('_x264_') | 
                                 ~test_dataset.video_path.str.contains('_x264-')) &
                                (test_dataset.video_path.str.contains('resized'))]"""

"train_dataset = train_dataset.loc[\n                                  (train_dataset.video_path.str.contains('_x264_') |\n                                   ~train_dataset.video_path.str.contains('_x264-')) & \n                                  (train_dataset.video_path.str.contains('resized'))]\n\ntest_dataset = test_dataset.loc[\n                                (test_dataset.video_path.str.contains('_x264_') | \n                                 ~test_dataset.video_path.str.contains('_x264-')) &\n                                (test_dataset.video_path.str.contains('resized'))]"

In [17]:
test_dataset.shape

(400, 3)

In [18]:
train_dataset.shape

(1600, 3)

In [19]:
"""train_dataset.to_csv(PATH_BLUR_CONVLSTM + 'train_uniform_sample_rwf2000.txt', index=False, header=None, sep=';')
test_dataset.to_csv(PATH_BLUR_CONVLSTM + 'test_uniform_sample_rwf2000.txt', index=False, header=None, sep=';')"""

"train_dataset.to_csv(PATH_BLUR_CONVLSTM + 'train_uniform_sample_rwf2000.txt', index=False, header=None, sep=';')\ntest_dataset.to_csv(PATH_BLUR_CONVLSTM + 'test_uniform_sample_rwf2000.txt', index=False, header=None, sep=';')"

In [20]:
train_dataset

,video_path,label,new_path
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,NaN
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,NaN
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,NaN
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,NaN
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,NaN
...,...,...,...
1595,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,NaN
1596,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,NaN
1597,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,NaN
1598,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,NaN


In [21]:
max_batch_size = 5

train_dataset_list = train_dataset.new_path.tolist()
train_label_list = train_dataset.label.tolist()

test_dataset_list = test_dataset.new_path.tolist()
test_label_list = test_dataset.label.tolist()
NUM_WORKERS=4
EPOCHS=60
BATCH_SIZE=8

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

STEPS_PER_EPOCH = int(len(train_dataset_list) / BATCH_SIZE)
VAL_STEPS_PER_EPOCH = int(len(test_dataset_list) / BATCH_SIZE)

shapes = ((BATCH_SIZE, SequenceLength, IMSIZE[0], IMSIZE[1], 3), (BATCH_SIZE, N_CLASSES))
dtypes = (tf.float32, tf.float32)

# Treino

In [22]:
from keras.models import Sequential, Input, Model
from keras.models import load_model
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation, LeakyReLU, Add, Multiply
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import mixed_precision

In [23]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [24]:
# def load_pretrained_model():
#   sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

#   model = load_model('/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/keras_model.h5')

#   model.compile(optimizer=sgd,
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])
  
#   return model

## Model 0

In [25]:
def create_model(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)


  ##############################################
  ##############################################

  #https://github.com/satyachaurasia/Violence-Detection-using-ConvLSTM/blob/master/ViolentModel.py
  #file:///C:/Users/jayne/Documents/Mestrado/Artigos%20coletados/A%20Frame-Based%20Feature%20Model%20for%20Violence%20Detection%20from%20Surveillance%20Cameras%20Using%20ConvLSTM%20Network.pdf
  #https://github.com/CankayaUniversity/ceng-407-408-2020-2021-Violent-Activity-Detection-from-Videos/blob/main/model.ipynb
  #https://www.programcreek.com/python/example/120287/keras.layers.ConvLSTM2D

  #3 blocos ConvLSTM + Fully conected

  lstm = ConvLSTM2D(filters=32, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(rgb)
  lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)

  #lstm = Dropout(0.4)(lstm)
  lstm = BatchNormalization()(lstm)

  lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(lstm)
  lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
  lstm = BatchNormalization()(lstm)

  lstm = ConvLSTM2D(filters=128, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(lstm)
  lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)

  lstm = BatchNormalization()(lstm)

  ##############################################
  ##############################################


  #Model 
  #lstm = ConvLSTM2D(filters=256, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(inputs)


  ##################################################### Fusion and Pooling
  #x = Multiply()([rgb, lstm])
  x = MaxPooling3D(pool_size=(8,1,1))(lstm)

  ##################################################### Merging Block

  """lstm = ConvLSTM2D(filters=256, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
  lstm = Flatten()(lstm)
  dropout = 0.5
  lstm = BatchNormalization()(lstm)
  lstm = Dropout(dropout)(lstm)
  lstm = Dense(1000)(lstm)
  lstm = Activation('relu')(lstm)
  lstm = Dense(256)(lstm)
  lstm = Dropout(dropout)(lstm)
  lstm = Activation('relu')(lstm)"""


  
  """x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)"""

  #####################################################
  x = Flatten()(x)


  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(N_CLASSES, activation='softmax', dtype='float32')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

## Model 2


In [26]:
def create_model1(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)


  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)


  #####################################################

  # cnn = TimeDistributed(cnn)(input_layer)
  lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
  #lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
  lstm = BatchNormalization()(lstm)

  lstm = ConvLSTM2D(filters=128, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
  #lstm = MaxPooling3D(pool_size=(2,2,2))(lstm)
  lstm = BatchNormalization()(lstm)

  lstm = ConvLSTM2D(filters=256, kernel_size=(3,3), strides=1, padding = "same", return_sequences=True, activation='relu')(x)
  lstm = BatchNormalization()(lstm)

  """  lstm = Flatten()(lstm)
  dropout = 0.5
  #lstm = BatchNormalization()(lstm)
  lstm = Dropout(dropout)(lstm)
  lstm = Dense(1000)(lstm)
  lstm = Activation('relu')(lstm)
  lstm = Dense(256)(lstm)
  lstm = Dropout(dropout)(lstm)
  lstm = Activation('relu')(lstm)"""


  #####################################################
  x = Flatten()(lstm)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(2, activation='softmax')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [27]:
model = create_model1()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 224, 224, 3)  0         
                             ]                                   
                                                                 
 conv3d_30 (Conv3D)          (None, 64, 224, 224, 16)  448       
                                                                 
 conv3d_31 (Conv3D)          (None, 64, 224, 224, 16)  784       
                                                                 
 max_pooling3d_21 (MaxPoolin  (None, 64, 112, 112, 16)  0        
 g3D)                                                            
                                                                 
 conv3d_32 (Conv3D)          (None, 64, 112, 112, 16)  2320      
                                                                 
 conv3d_33 (Conv3D)          (None, 64, 112, 112, 16)  784   

In [28]:
import tensorflow.keras as keras
from tensorflow.keras.utils import Sequence
from keras.utils import np_utils
import mediapy as media

class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, input_file, batch_size=1, shuffle=True, data_augmentation=True, uniform_sample=False):
        # Initialize the params
        self.batch_size = batch_size
        self.input_file = input_file
        self.classes = 2
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.uniform_sample = uniform_sample
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
        
    def search_data_v2(self):
        X_path = []
        Y_dict = {}
        """one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n')
              path = line[:-2]

              label = int(line[-1:])
              X_path.append(path)
              Y_dict[path] = one_hots[label]"""

        # # list all kinds of sub-folders
        # self.dirs = sorted(os.listdir(self.directory))
        # one_hots = np_utils.to_categorical(range(len(self.dirs)))
        # for i,folder in enumerate(self.dirs):
        #     folder_path = os.path.join(self.directory,folder)
        #     for file in os.listdir(folder_path):
        #         file_path = os.path.join(folder_path,file)
        #         # append the each file path, and keep its label  
        #         X_path.append(file_path)
        #         Y_dict[file_path] = one_hots[i]

        df_videos = pd.read_csv(self.input_file, header=None, sep=';', names=['video_path', 'label', 'frames32'])
        df_videos_filtered = df_videos.loc[df_videos.frames32 == 0]

        X_path = df_videos_filtered[['video_path', 'label']].video_path.to_list()
        Y_dict = pd.get_dummies(df_videos_filtered[['video_path', 'label']], columns=['label']).set_index('video_path').T.to_dict(orient="list")

        return X_path, Y_dict

    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n').split(';')
              path = line[0]

              label = int(line[1])
              X_path.append(path)
              Y_dict[path] = one_hots[label]

        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = self.classes
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(range(self.classes)):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
      
    def normalize(self, data):
        mean = np.mean(data)
        std = np.std(data)
        return (data-mean) / std
    
    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(m=video, axis=2)
        return video    
    
    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval 
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and 
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])     
        # calculate numer of padded frames and fix it 
        num_pad = target_frames - len(sampled_video)
        if num_pad>0:
            padding = [video[i] for i in range(-num_pad,0)]
            sampled_video += padding     
        # get sampled video

        #sampled_video = tf.image.resize(sampled_video, [224,224])
        return np.array(sampled_video, dtype=np.float32)
    
    def dynamic_crop(self, video):
        # extract layer of optical flow from video
        opt_flows = video[...,3]
        # sum of optical flow magnitude of individual frame
        magnitude = np.sum(opt_flows, axis=0)
        # filter slight noise by threshold 
        thresh = np.mean(magnitude)
        magnitude[magnitude<thresh] = 0
        # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
        x_pdf = np.sum(magnitude, axis=1) + 0.001
        y_pdf = np.sum(magnitude, axis=0) + 0.001
        # normalize PDF of x and y so that the sum of probs = 1
        x_pdf /= np.sum(x_pdf)
        y_pdf /= np.sum(y_pdf)
        # randomly choose some candidates for x and y 
        x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
        y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
        # get the mean of x and y coordinates for better robustness
        x = int(np.mean(x_points))
        y = int(np.mean(y_points))
        # avoid to beyond boundaries of array
        x = max(56,min(x,167))
        y = max(56,min(y,167))
        # get cropped video 
        return video[:,x-56:x+56,y-56:y+56,:]  
    
    def color_jitter(self,video):
        # range of s-component: 0-1
        # range of v component: 0-255
        s_jitter = np.random.uniform(-0.2,0.2)
        v_jitter = np.random.uniform(-30,30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
            s = hsv[...,1] + s_jitter
            v = hsv[...,2] + v_jitter
            s[s<0] = 0
            s[s>1] = 1
            v[v<0] = 0
            v[v>255] = 255
            hsv[...,1] = s
            hsv[...,2] = v
            video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        return video
        
    def load_data(self, path):
        data = media.read_video(path)[...,:3]
        data = np.float32(data)
        # sampling 64 frames uniformly from the entire video

        if self.uniform_sample:
          data = self.uniform_sampling(video=data, target_frames=64)
        else:
          data = np.array(data, dtype=np.float32)

        # whether to utilize the data augmentation
        if  self.data_aug:
            data = self.color_jitter(data)
            data = self.random_flip(data, prob=0.5)
        # normalize

        data = self.normalize(data)
        return data

In [29]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    # Every 10 epochs, the learning rate is reduced to 1/10 of the original
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.5)
    return K.get_value(model.optimizer.lr)

In [30]:
train_generator = DataGenerator(input_file='/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train_uniform_sample_rwf2000.txt', 
                            batch_size=BATCH_SIZE, 
                            data_augmentation=True,
                            uniform_sample=False)
  

Found 1600 files belonging to 2 classes.
         0 :  0
         1 :  1


In [31]:
batch = next(iter(train_generator))

In [32]:
batch[0][0].shape

(64, 224, 224, 3)

In [33]:
with tf.device('/gpu:0'):  
  # train_pipe = video_pipe(
  #     file_list=train_dataset_list,
  #     label_list=train_label_list,
  #     batch_size=BATCH_SIZE, 
  #     num_threads=4
  # )

  # train_generator = dali_tf.DALIDataset(
  #       pipeline=train_pipe,
  #       batch_size=BATCH_SIZE,
  #       output_shapes=shapes,
  #       output_dtypes=dtypes,
  #       device_id=0
  # )

  # test_pipe = video_pipe(
  #     file_list=test_dataset_list,
  #     label_list=test_label_list,
  #     batch_size=BATCH_SIZE, 
  #     num_threads=4
  # )

  # test_generator = dali_tf.DALIDataset(
  #       pipeline=test_pipe,
  #       batch_size=BATCH_SIZE,
  #       output_shapes=shapes,
  #       output_dtypes=dtypes,
  #       device_id=0 
  # )

  train_generator = DataGenerator(input_file='/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train_uniform_sample_rwf2000.txt', 
                              batch_size=BATCH_SIZE, 
                              data_augmentation=True,
                              uniform_sample=False)
    
  val_generator = DataGenerator(input_file='/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/test_uniform_sample_rwf2000.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)

  
  model = create_model1()

  reduce_lr = LearningRateScheduler(scheduler)
  tb_callback = tf.keras.callbacks.TensorBoard(PATH_TENSORBOARD, update_freq=1)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=PATH_BLUR_CONVLSTM_MODEL_CHECKPOINTS + "model.{epoch:02d}.h5",
                                                  save_weights_only=False,
                                                  verbose=1)
  
  callbacks_list = [reduce_lr, tb_callback, cp_callback]
  
  hist = model.fit(
    train_generator, 
    validation_data=val_generator,
    validation_steps=VAL_STEPS_PER_EPOCH,
    callbacks=callbacks_list,
    workers=16,
    max_queue_size=8, 
    verbose=1, 
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,)

Found 1600 files belonging to 2 classes.
         0 :  0
         1 :  1
Found 400 files belonging to 2 classes.
         0 :  0
         1 :  1
Epoch 1/60
  7/200 [>.............................] - ETA: 3:50 - loss: 44.3152 - precision_1: 0.5357 - categorical_accuracy: 0.5357 - recall_1: 0.5357 - auc_1: 0.5408

InvalidArgumentError: ignored

In [ ]:
with tf.device('/gpu:0'):  
  # train_pipe = video_pipe(
  #     file_list=train_dataset_list,
  #     label_list=train_label_list,
  #     batch_size=BATCH_SIZE, 
  #     num_threads=4
  # )

  # train_generator = dali_tf.DALIDataset(
  #       pipeline=train_pipe,
  #       batch_size=BATCH_SIZE,
  #       output_shapes=shapes,
  #       output_dtypes=dtypes,
  #       device_id=0
  # )

  # test_pipe = video_pipe(
  #     file_list=test_dataset_list,
  #     label_list=test_label_list,
  #     batch_size=BATCH_SIZE, 
  #     num_threads=4
  # )

  # test_generator = dali_tf.DALIDataset(
  #       pipeline=test_pipe,
  #       batch_size=BATCH_SIZE,
  #       output_shapes=shapes,
  #       output_dtypes=dtypes,
  #       device_id=0 
  # )

  train_generator = DataGenerator(input_file=PATH_BLUR_CONVLSTM + 'train_uniform_sample_rwf2000.txt', 
                              batch_size=BATCH_SIZE, 
                              data_augmentation=True,
                              uniform_sample=False)
    
  val_generator = DataGenerator(input_file=PATH_BLUR_CONVLSTM + 'test_uniform_sample_rwf2000.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)

  
  model = create_model1()

  reduce_lr = LearningRateScheduler(scheduler)
  tb_callback = tf.keras.callbacks.TensorBoard(PATH_TENSORBOARD, update_freq=1)
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=PATH_BLUR_CONVLSTM_MODEL_CHECKPOINTS + "model.{epoch:02d}.h5",
                                                  save_weights_only=False,
                                                  verbose=1)
  
  callbacks_list = [reduce_lr, tb_callback, cp_callback]
  
  hist = model.fit(
    train_generator, 
    validation_data=val_generator,
    validation_steps=VAL_STEPS_PER_EPOCH,
    callbacks=callbacks_list,
    workers=16,
    max_queue_size=8, 
    verbose=1, 
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,)

In [ ]:
def load_pretrained_model():
  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model = load_model('/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline_mixed_precision - rwf/checkpoints/model.30.h5')

  model.compile(optimizer=sgd,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model
  
model = load_pretrained_model()
model.summary()

In [ ]:
train_iter = iter(train_generator)
test_iter = iter(test_generator)

In [ ]:
train_batch = next(train_iter)
videos, labels = train_batch
mediapy.show_videos(videos)

print(labels)

In [ ]:
videos

In [ ]:
videos, labels = next(test_iter)

mediapy.show_videos(videos.numpy())
print(labels)

In [ ]:
labels

In [ ]:
labels

In [ ]:
%load_ext tensorboard

In [ ]:
PATH_TENSORBOARD

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model/runs/